Initial Setup and Review (Run First)

This initial block reloads the sample medical image (brain slice) and defines some helper functions and global variables (`M_c5`, `N_c5`, `P_c5`, `Q_c5`) that are used in the examples below.


How the Code Works:

* Imports: Loads the required libraries.
* Image Loading: Loads `data.brain()` and selects a central slice (or uses `camera` as fallback).
* Initial Definitions:
  * `idft_process_extract`: helper for inverse DFT with padding removal.
  * `plot_images_c5`: helper for displaying multiple images.
* Degradation Model Review: prints the conceptual equations in spatial and frequency domains.


In [ ]:
# --- Required Imports ---
import numpy as np
import matplotlib.pyplot as plt
from skimage import data, img_as_float, img_as_ubyte, exposure, util, filters
from scipy import ndimage
from numpy.fft import fft2, ifft2, fftshift, ifftshift
import time # Not actively used in these examples, but useful to keep

# --- Load Medical Image and Define Global Variables ---
try:
    brain_volume = data.brain()
    if brain_volume.ndim == 3:
        slice_index = brain_volume.shape[0] // 2
        image_gray_orig_c5 = brain_volume[slice_index, :, :]
    elif brain_volume.ndim == 2:
        image_gray_orig_c5 = brain_volume
        slice_index = "N/A (2D Image)"
    else:
        raise ValueError("Unexpected 'brain' image format.")
    print(f"'brain' image (slice {slice_index if brain_volume.ndim == 3 else ''}) loaded for Ch. 5.")
except Exception as e:
    print(f"Error loading 'brain': {e}. Using 'camera' as fallback.")
    image_gray_orig_c5 = data.camera()

image_float_c5 = img_as_float(image_gray_orig_c5.copy())
M_c5, N_c5 = image_float_c5.shape # Original dimensions
P_c5, Q_c5 = 2*M_c5, 2*N_c5     # Padded dimensions

# --- Helper Function for IDFT, Extraction, and Rescaling ---
def idft_process_extract(G_centralizado, P_img, Q_img, M_orig, N_orig):
    """Applies ifftshift/ifft2, takes real part, removes padding, and rescales."""
    G_canto_dc = ifftshift(G_centralizado) # Move DC to the corner before IDFT
    img_padded_spatial = ifft2(G_canto_dc).real
    img_final = img_padded_spatial[0:M_orig, 0:N_orig] # Remove padding
    return exposure.rescale_intensity(img_final, out_range=(0,1))

# --- Helper Plot Function (already defined earlier, repeated here for self-contained execution) ---
def plot_images_c5(images, titles, cmaps=None, rows=1, cols=None, figsize=(15,5)):
    num_images = len(images)
    if cols is None:
        cols = (num_images + rows - 1) // rows
    fig, axes = plt.subplots(rows, cols, figsize=figsize, squeeze=False)
    axes_flat = axes.ravel()
    if cmaps is None: cmaps_list = ['gray'] * num_images
    elif isinstance(cmaps, str): cmaps_list = [cmaps] * num_images
    else:
        cmaps_list = list(cmaps)
        if len(cmaps_list) < num_images: cmaps_list.extend(['gray']*(num_images-len(cmaps_list)))
    for i in range(len(axes_flat)):
        if i < num_images:
            img, title, cmap = images[i], titles[i], cmaps_list[i]
            axes_flat[i].imshow(img, cmap=cmap if img.ndim==2 else None); axes_flat[i].set_title(title); axes_flat[i].axis('off')
        else: axes_flat[i].axis('off')
    plt.tight_layout(); plt.show()

# Degradation Model (Conceptual Review)
print("\n--- Degradation/Restoration Model (Review) ---")
print("Spatial Domain: g(x,y) = h(x,y) * f(x,y) + η(x,y)")
print("Frequency Domain: G(u,v) = H(u,v)F(u,v) + N(u,v)")
print("f: original, h/H: degradation, η/N: noise, g/G: observed")

plot_images_c5([image_float_c5], ["Original Test Image (f)"], cmaps=['gray'], figsize=(5,5))

The text output confirms the degradation model.

The displayed image is the MRI slice (or the `camera` image as fallback), which will be used as `f(x,y)` in the restoration examples.


5.4 Periodic Noise Reduction by Frequency-Domain Filtering


This example simulates adding periodic (sinusoidal) noise to an image and then applies a notch filter in the frequency domain to remove it.


1. Periodic Noise Addition: A sinusoidal noise with `num_noise_cycles` cycles along image height `M_c5` is added to `image_float_c5`.
2. DFT and Spectrum: The noisy image is padded and transformed into the frequency domain.
3. Notch Filter Construction: Two Gaussian notches are placed at the expected noise peak locations.
4. Filtering and IDFT: The filter is applied in frequency domain, then transformed back to spatial domain.


In [ ]:
# 5.4 Periodic Noise Reduction

print("\n--- Example 5.4: Periodic Noise Reduction ---")
# 1. Add sinusoidal periodic noise to the original image
img_with_periodic_noise = image_float_c5.copy()
num_noise_cycles = 20 # Number of vertical noise cycles
noise_period_px_5_4 = M_c5 / num_noise_cycles
noise_amplitude_5_4 = 0.3
for r in range(M_c5):
    img_with_periodic_noise[r,:] += noise_amplitude_5_4 * np.cos(2 * np.pi * r / noise_period_px_5_4)
img_with_periodic_noise = np.clip(img_with_periodic_noise, 0, 1)

# 2. Prepare noisy image for DFT (padding)
fp_noisy_notch_pad = np.zeros((P_c5, Q_c5), dtype=float)
fp_noisy_notch_pad[0:M_c5, 0:N_c5] = img_with_periodic_noise

# 3. Compute DFT and center the spectrum
Fp_noisy_notch_centered = fftshift(fft2(fp_noisy_notch_pad))
noisy_spectrum_vis = np.log1p(np.abs(Fp_noisy_notch_centered))

# 4. Create frequency grids (U for vertical, V for horizontal in the spectrum)
II_filt_5_4, JJ_filt_5_4 = np.indices((P_c5, Q_c5))
U_map_filt_5_4 = II_filt_5_4 - P_c5//2
V_map_filt_5_4 = JJ_filt_5_4 - Q_c5//2

# 5. Locate noise peaks
kv_noise_peak_5_4 = num_noise_cycles * (P_c5 / M_c5) # Peak position on U_map axis

# 6. Notch Filter Construction
sigma_notch_5_4 = P_c5 * 0.01 # Notch width (e.g., 1% of P). Adjust as needed.
H_filtro_notch_5_4 = np.ones((P_c5, Q_c5), dtype=float)

if kv_noise_peak_5_4 != 0:
    # Peak 1: (U_map = +kv_pico_ruido, V_map = 0)
    D1_sq = (U_map_filt_5_4 - kv_noise_peak_5_4)**2 + (V_map_filt_5_4 - 0)**2
    H_filtro_notch_5_4 *= (1 - np.exp(-D1_sq / (2 * sigma_notch_5_4**2)))
    # Peak 2: (U_map = -kv_pico_ruido, V_map = 0)
    D2_sq = (U_map_filt_5_4 + kv_noise_peak_5_4)**2 + (V_map_filt_5_4 - 0)**2
    H_filtro_notch_5_4 *= (1 - np.exp(-D2_sq / (2 * sigma_notch_5_4**2)))

# 7. Apply notch filter
G_notch_filtered_centered = H_filtro_notch_5_4 * Fp_noisy_notch_centered

# 8. IDFT and post-processing
img_notch_filtered = idft_process_extract(G_notch_filtered_centered, P_c5, Q_c5, M_c5, N_c5)

# Visualization
plot_images_c5([img_with_periodic_noise, noisy_spectrum_vis, H_filtro_notch_5_4, img_notch_filtered],
               [f"With Periodic Noise ({num_noise_cycles} cycles)", "Noisy Spectrum", f"Notch Filter (sigma={sigma_notch_5_4:.1f})", "Notch Result"],
               rows=2, cols=2, cmaps=['gray','viridis','viridis','gray'], figsize=(10,10))

* Image with Periodic Noise: The first image shows horizontal stripe-like sinusoidal artifacts.
* Noisy Spectrum: The second image shows bright symmetric peaks away from the center, corresponding to the periodic noise.
* Notch Filter: The third image shows attenuation at exactly those peak locations.
* Notch Result: The fourth image should show reduced periodic artifacts with better overall quality.


5.5 Linear and Position-Invariant Degradations


This section is mainly conceptual and provides the foundation for restoration filters that handle blur. There is no practical filtering example here.


How the Code Works (Conceptual):

It reviews that if degradation `h(x,y)` is Linear and Position-Invariant (LPI), then degraded image `g(x,y)` is given by convolution with the original `f(x,y)` (ignoring noise), and multiplication in frequency domain.


In [ ]:
# 5.5 Linear and Position-Invariant Degradations (Conceptual)
print("\n--- Section 5.5: Linear and Position-Invariant Degradations (LPI) ---")
print("This section is fundamentally theoretical, establishing that if degradation h(x,y) is LPI,")
print("then g(x,y) = h(x,y) * f(x,y) (without noise).")
print("In frequency domain: G(u,v) = H(u,v)F(u,v).")
print("This model is the basis for the deconvolution filters shown next.")
# There is no specific code to run here; this cell reinforces the concept.

5.6 Estimating the Degradation Function H


To restore an image degraded by blur, we need an estimate of the degradation function `H(u,v)` (or its spatial PSF `h(x,y)`).


1. Motion PSF Simulation:
* `L_mov = 20`: motion blur length in pixels.
* `theta_mov = 0`: horizontal motion angle.
* A motion PSF is created and then padded/rolled to match DFT convention.

2. `H(u,v)` Estimation:
* The FFT of padded PSF gives `H_motion_not_centered` (DC at corner).
* `fftshift` is used for visualization.

3. Degradation Simulation:
* Original image (padded) is multiplied by `H(u,v)` in frequency domain.
* Inverse FFT gives a blurred image (motion blur).


In [ ]:
# M_c5, N_c5 are the dimensions of the original image (ex: image_float_c5)
# P_c5, Q_c5 are the padded dimensions (ex: 2*M_c5, 2*N_c5)

# 1. Create a zero matrix with padded dimensions (P_c5 x Q_c5)
fp_para_filtragem = np.zeros((P_c5, Q_c5), dtype=float)

# 2. Copy the original image to the top-left corner of this larger matrix
fp_para_filtragem[0:M_c5, 0:N_c5] = image_float_c5

In [ ]:
# 5.6 Estimating the Degradation Function H

print("\n--- Example 5.6: Simulation of a Motion PSF and its H(u,v) ---")
# 1. Simulate a horizontal motion-blur PSF
L_mov = 21 # Motion length (pixels), odd for a defined center
psf_motion = np.zeros((L_mov, L_mov), dtype=float)
# Create a horizontal line at PSF center
psf_motion[L_mov//2, :] = 1.0
psf_motion /= np.sum(psf_motion) # Normaliza a PSF

# 2. Compute H(u,v) from PSF (with padding and centering for visualization)
psf_motion_padded = np.zeros((P_c5, Q_c5), dtype=float)
# Place PSF in top-left corner for DFT (convolution origin)
# However, to keep H(u,v) real after shift (if PSF is symmetric),
# a PSF deve estar centralizada no array antes do fftshift(fft2(...))
# place PSF in the corner and shift H(u,v) for visualization.
# For degradation computation G = FH, H must follow the same ordering as F.
# If F has DC at corner, H must too. If F has centered DC, H must too.

# Standardization: PSF origin at corner (0,0) of PxQ array
# The PSF center (pixel L_mov//2, L_mov//2) maps to (0,0) in PxQ after roll.
# Alternatively, place the PSF directly at corner (0,0) of PxQ array.
# psf_motion_padded[0:L_mov, 0:L_mov] = psf_motion # PSF at corner

# For frequency-domain convolution G = F*H to work correctly,
# H must be the DFT of PSF h(x,y) with origin at pixel (0,0) of the matrix.
# A PSF que criamos (psf_motion) tem seu "center" em L_mov//2.
# We place the psf_motion center at (0,0) of psf_motion_padded,
# allowing wraparound, which is correct for DFT/convolution.
psf_motion_padded_rolled = np.zeros((P_c5, Q_c5), dtype=float)
r_offset, c_offset = L_mov//2, L_mov//2
for r_psf in range(L_mov):
    for c_psf in range(L_mov):
        # Map (r_psf, c_psf) from PSF to PxQ positions with (0,0) as PSF center
        # (r_psf - r_offset) mod P_c5 etc.
        idx_r_pad = (r_psf - r_offset + P_c5) % P_c5
        idx_c_pad = (c_psf - c_offset + Q_c5) % Q_c5
        psf_motion_padded_rolled[idx_r_pad, idx_c_pad] = psf_motion[r_psf, c_psf]

H_motion_not_centered = fft2(psf_motion_padded_rolled) # DC at corner
H_motion_centered_vis = fftshift(H_motion_not_centered) # centered DC for visualization

# 3. Simulate Degradation on Original Image
# fp_para_filtragem was defined in 4.7/4.8 as padded image_float
F_img_original_padded_not_centered = fft2(fp_para_filtragem) # DC at corner

# G = F * H (both with DC at corner)
G_motion_degraded_not_centered = F_img_original_padded_not_centered * H_motion_not_centered
img_motion_degraded_padded = ifft2(G_motion_degraded_not_centered).real
img_motion_degraded = exposure.rescale_intensity(img_motion_degraded_padded[0:M_c5, 0:N_c5], out_range=(0,1))

# Visualization
plot_images_c5([psf_motion, np.log1p(np.abs(H_motion_centered_vis)), image_float_c5, img_motion_degraded],
               [f"Motion PSF ({L_mov}px horiz.)", "H_mov Spectrum (Log)", "Original Image", "Motion-Degraded Image"],
               rows=2, cols=2, cmaps=['gray','viridis','gray','gray'], figsize=(10,10))

Interpreting Results (5.6):

* Motion PSF: The first image shows the PSF that simulates horizontal motion blur.
* `|H(u,v)|` Spectrum: The second image shows periodic nulls/attenuations where frequencies are suppressed by motion blur.
* Degraded Image: The third image shows characteristic directional blur from linear motion.


5.7 Inverse Filtering


Inverse filtering attempts to reverse degradation by dividing the degraded image DFT by the degradation function DFT `H(u,v)`. This section shows direct inverse filtering and truncated inverse filtering.


1. Degraded Image and `H(u,v)`: Reuses `img_motion_degraded` and `H_motion_not_centered` from the previous example; adds Gaussian noise to make the restoration challenge realistic.
2. Direct Inverse Filtering: Computes `F_hat = G / H` with epsilon for numerical stability.
3. Truncated Inverse Filtering (Pseudo-inverse): Uses inverse filtering only where `|H|` is above a threshold.
4. Comparison: Plots degraded input, direct inverse result, and truncated inverse result.


In [ ]:
# 5.7 Inverse Filtering

print("\n--- Example 5.7: Inverse Filtering ---")
# Use the motion-degraded image and H(u,v) from previous example (5.6)
# Assumes variables image_float_c5, M_c5, N_c5, P_c5, Q_c5,
# H_motion_not_centered (DFT of motion PSF, DC at corner),
# and img_motion_degraded (blurred image without extra noise) are defined from Example 5.6.

# Add a bit of noise to the motion-degraded image
sigma_noise_inv_5_7 = 0.01
img_motion_degraded_ruidosa_5_7 = util.random_noise(img_motion_degraded, mode='gaussian', var=sigma_noise_inv_5_7**2)
img_motion_degraded_ruidosa_5_7 = np.clip(img_motion_degraded_ruidosa_5_7, 0, 1)

# DFT of degraded noisy image (with implicit padding in PxQ dimensions)
fp_degraded_noisy_5_7 = np.zeros((P_c5, Q_c5))
fp_degraded_noisy_5_7[0:M_c5, 0:N_c5] = img_motion_degraded_ruidosa_5_7
G_degraded_noisy_not_cent_5_7 = fft2(fp_degraded_noisy_5_7) # DC at corner

# Define epsilon
epsilon = 1e-8 # Small constant to avoid division by zero

# 1. Direct Inverse Filtering
# H_motion_not_centered is the PSF DFT (HxQ) with DC at corner
F_hat_inverse_direct_not_cent_5_7 = G_degraded_noisy_not_cent_5_7 / (H_motion_not_centered + epsilon)
img_rest_inverse_direct_padded_5_7 = ifft2(F_hat_inverse_direct_not_cent_5_7).real
img_rest_inverse_direct_5_7 = exposure.rescale_intensity(img_rest_inverse_direct_padded_5_7[0:M_c5, 0:N_c5], out_range=(0,1))

# 2. Truncated Inverse Filtering (Pseudo-inverse)
H_abs_not_cent_5_7 = np.abs(H_motion_not_centered)
H_truncation_threshold_5_7 = H_abs_not_cent_5_7.max() * 0.05 # e.g., 5% of max |H|

F_hat_inverse_trunc_not_cent_5_7 = np.zeros_like(G_degraded_noisy_not_cent_5_7, dtype=complex)
H_valid_mask_5_7 = H_abs_not_cent_5_7 > H_truncation_threshold_5_7

# Avoid divide-by-zero warnings even with epsilon by checking mask first
# and ensuring H_motion_not_centered[H_valid_mask_5_7] is not zero.
# Epsilon already helps, but this check is safer.
H_valid = H_motion_not_centered[H_valid_mask_5_7]
G_valid = G_degraded_noisy_not_cent_5_7[H_valid_mask_5_7]
F_hat_inverse_trunc_not_cent_5_7[H_valid_mask_5_7] = G_valid / (H_valid + epsilon)


img_rest_inverse_trunc_padded_5_7 = ifft2(F_hat_inverse_trunc_not_cent_5_7).real
img_rest_inverse_trunc_5_7 = exposure.rescale_intensity(img_rest_inverse_trunc_padded_5_7[0:M_c5, 0:N_c5], out_range=(0,1))

# Visualization
plot_images_c5([img_motion_degraded_ruidosa_5_7, img_rest_inverse_direct_5_7, img_rest_inverse_trunc_5_7],
               ["Motion-Degraded + Noise", "Direct Inverse Restoration", f"Truncated Inverse Restoration (limiar={H_truncation_threshold_5_7:.2e})"],
               cmaps=['gray','gray','gray'])

* Degraded by Motion + Noise: Input image for restoration.
* Direct Inverse Restoration: Usually amplifies noise strongly, especially where `|H|` is close to zero.
* Truncated Inverse Restoration: Generally more stable by avoiding inversion where `H` is too small, reducing severe artifacts.


5.8 Wiener Filtering


Wiener filtering is a statistical restoration approach that minimizes mean squared error between original and restored images.


1. Inputs:
* `G_degradada_ruidosa_nao_cent`: DFT of degraded noisy image (DC at corner).
* `H_motion_not_centered`: DFT of motion PSF.

2. Wiener Filter Equation:
* Uses `F_hat = [H* / (|H|^2 + K)] * G`, where `K` approximates noise-to-signal power ratio.

3. Output:
* Reconstructs restored image and compares with truncated inverse filtering.


In [ ]:
# 5.8 Wiener Filtering

print("\n--- Example 5.8: Wiener Filtering ---")
# Reuse G_degraded_noisy_not_cent_5_7 (DFT da imagem degradada+noise, DC at corner) do Example 5.7
# Reuse H_motion_not_centered (DFT of PSF, DC at corner) do Example 5.6

# Define epsilon
epsilon = 1e-8 # Small constant to avoid division by zero

# K parameter for Wiener filter (approximation of Sn(u,v)/Sf(u,v))
K_wiener_5_8 = 0.01 # Tune this value experimentally (ex: 0.1, 0.01, 0.001)

# Wiener filter components (Eq. 5.8-7: F_hat = [H* / (|H|^2 + Sn/Sf)] * G )
# Sn/Sf is approximated by K.
H_conj_wiener_5_8 = np.conj(H_motion_not_centered)
H_mag_sq_wiener_5_8 = np.abs(H_motion_not_centered)**2

# The K_wiener_5_8 term in denominator prevents division by zero when H_mag_sq_wiener_5_8 is zero.
Wiener_filter_active_part_5_8 = H_conj_wiener_5_8 / (H_mag_sq_wiener_5_8 + K_wiener_5_8 + epsilon)
F_hat_wiener_not_cent_5_8 = Wiener_filter_active_part_5_8 * G_degraded_noisy_not_cent_5_7

# IDFT, remove padding, and rescale
# (Assuming idft_process_extract was defined in the Initial Setup cell)
img_rest_wiener_padded_5_8 = ifft2(F_hat_wiener_not_cent_5_8).real
img_rest_wiener_5_8 = exposure.rescale_intensity(img_rest_wiener_padded_5_8[0:M_c5, 0:N_c5], out_range=(0,1))

# Visualization
# Reuse img_rest_inverse_trunc_5_7 for comparison
plot_images_c5([img_motion_degraded_ruidosa_5_7, img_rest_inverse_trunc_5_7, img_rest_wiener_5_8],
               ["Motion-Degraded + Noise", "Truncated Inverse Restoration (ref.)", f"Wiener Restoration (K={K_wiener_5_8})"],
               cmaps=['gray','gray','gray'])

* Degraded by Motion + Noise: Input image.
* Truncated Inverse Restoration (ref.): Reference from previous section.
* Wiener Restoration: Usually achieves a better balance between deblurring and noise suppression.
